In [1]:
import numpy as np

In [2]:
def binomial_tree_option(S, K, T, r, sigma, n, option_type):
    # Paso 1: Calcular los factores de subida (u) y bajada (d)
    # y la probabilidad (p)
    dt = T/n  # dt es el tamaño del paso de tiempo
    u = np.exp(sigma * np.sqrt(dt))
    print('u:', u)
    d = 1/u
    print('d: ', d)
    q = (np.exp(r * dt) - d) / (u - d)
    print('q: ', q)

    stock_tree = np.zeros((n+1, n+1))
    option_tree = np.zeros((n+1, n+1))


    for i in range(n+1):
        for j in range(i+1):
            stock_tree[j, i] = S * (u ** (i - j)) * (d ** j)

    # Paso 3: Crear el árbol de valores de la opción

    # Primero estima los valores intrínsecos al vencimiento
    if option_type == 'call':
        option_tree[:, n] = np.maximum(np.zeros(n+1), stock_tree[:, n] - K)
    elif option_type == 'put':
        option_tree[:, n] = np.maximum(np.zeros(n+1), K - stock_tree[:, n])

    # Luego, Retroceder a través del árbol de opciones
    for i in range(n-1, -1, -1):
        for j in range(i+1):
            option_tree[j, i] = np.exp(-r * dt) * (q * option_tree[j, i+1] + (1-q) * option_tree[j+1, i+1])

            # Opción americana
            if option_type == 'call':
                option_tree[j, i] = np.maximum(option_tree[j, i], stock_tree[j, i] - K)
            elif option_type == 'put':
                option_tree[j, i] = np.maximum(option_tree[j, i], K - stock_tree[j, i])

    #Regresa el valor estimado en la raíz, y los árboles generados
    return option_tree[0, 0], stock_tree, option_tree

# Parámetros de ejemplo
S = 50      # Precio del activo subyacente
K = 50      # Precio de ejercicio
T = 1       # Tiempo hasta la expiración (años)
r = 0.05    # Tasa de interés libre de riesgo (anual)
sigma = 0.10    # Volatilidad (anual)
n = 365      # Número de pasos en el árbol binomial

# Calcular el precio de la opción americana de compra (call)
call_option_price, stock_tree, option_tree = binomial_tree_option(S, K, T, r, sigma, n, option_type='call')
print(f"El precio de la opción americana de compra es: {call_option_price:.9f}")


# Calcular el precio de la opción americana de venta (put)
put_option_price, stock_tree, option_tree = binomial_tree_option(S, K, T, r, sigma, n, option_type='put')
print(f"El precio de la opción americana de venta es: {put_option_price:.9f}")

u: 1.005247961787983
d:  0.9947794355348418
q:  0.5117778778051291
El precio de la opción americana de compra es: 3.403358720
u: 1.005247961787983
d:  0.9947794355348418
q:  0.5117778778051291
El precio de la opción americana de venta es: 1.218766220


In [3]:
def american_option_price(So, K, T, rf, sigma, n, option_type='call'):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    disc = np.exp(-rf * dt)
    p = (np.exp(rf * dt) - d) / (u - d)
    j = np.arange(n + 1)
    ST = So * (u ** (n - j)) * (d ** j)

    option_values = np.maximum(K - ST, 0.0) if option_type == 'put' else np.maximum(ST - K, 0.0)

    for i in range(n - 1, -1, -1):
        ST = ST[:-1] / u
        option_values = disc * (p * option_values[:-1] + (1 - p) * option_values[1:])
        exercise = np.maximum(K - ST, 0.0) if option_type == 'put' else np.maximum(ST - K, 0.0)
        option_values = np.maximum(option_values, exercise)

    return option_values[0]

In [4]:
So = 50
K = 50
T = 1
rf = 0.05
sigma = 0.10
n = 365

call_price = american_option_price(So, K, T, rf, sigma, n, option_type='call')
put_price = american_option_price(So, K, T, rf, sigma, n, option_type='put')
print(f"El precio de la opción americana de compra es: {call_price:.9f}")
print(f"El precio de la opción americana de venta es: {put_price:.9f}")

El precio de la opción americana de compra es: 3.403358720
El precio de la opción americana de venta es: 1.218766220
